# PRÁCTICA 1 V4 - ALGORITMO BASADOS EN ENTORNOS Y TRAYECTORIAS

# Funciones Base

## Carga de datos

In [64]:
import numpy as np
import pandas as pd

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')

In [65]:
def movements_to_list(deltas_df):
    """
    Devuelve array con todos los movimientos realizados en las estaciones. Cada posicion del array esta compuesto por el id de la estacion y la cantidad de bicis retiradas/aniadidas en cada movimiento
    
    Parametros
        deltas_df : DataFrame
            Pandas DataFrame a convertir
    Return
        Numpy array
    """
    move_list = np.array([]) # Inicializamos el array
    rows = len(deltas_df.index) # Mediciones totales realizadas
    columns = len(deltas_df.columns) # Numero de estaciones
    
    # Recorreremos todas las mediciones e iremos transformando todas ellas que sean distintas de 0
    for i in range(0, rows):
        if i != 0:
            row = deltas_df.iloc[i] # Seleccionamos una medicion
            selec = row[row != 0] # Seleccionamos que estaciones han sufrido algun cambio
            for j in range(0, len(selec)):
                lst = np.array([[int(selec.index[j]), selec[j]]]) # Obtenemos un array con el index y el numero de bicis retiradas/aniadidas
                # En la setencia anterior, el primer corchete sirve para crear el array. Mientras que el segundo para crear una tupla tal que: [estacion, movements]
                if len(move_list) == 0:
                    move_list = lst # Asignamos la primera tupla como 
                else:
                    move_list = np.append(move_list, lst, axis=0) # Aniadimos una nueva fila(axis=0) a move_list
    
    return move_list

In [66]:
def update_dataframe(df, df_columns, seed, algorithm, km, t, solution):
    """
    Actualiza un data frame ya creado
    
    Parametros
        df : Pandas DataFrame
            DataFrame a actualizar
        df_columns : list
            Nombre de las columnas del DataFrame
        seed : int
            Semilla utilizada en el algoritmo
        algorithm : str
            Nombre del algoritmo utilizado
        km : float
            Kilometros obtenidos por el algoritmo
        t : float
            Tiempo demorado por el agoritmo en ejecutarse
        solution : Numpy ndarray
            Vector solucion obtenido por el algoritmo
    
    Return 
        Devuelve una copia del DataFrame ya actualizado
    """
    
    data = [(seed, algorithm, km, t, solution)]
    new_df = pd.DataFrame(data, columns=df_columns)
    return df.append(new_df) # Le aniadimos a al dataframe la nueva fila y lo devolvemos

## Función de evaluación

In [67]:
def evaluate(move_list, init_state, capacity, index_m, kms_m):
    '''
    La funcion estipula el numero de kilometros recorrido por los usuarios al no encontrar una bicicleta en su estación de preferencia y tener que desplazarse a la estaion mas cercana con bicicletas disponibles, o la distancia recorrida en caso de no haber sitio para dejar la bicicleta en la estación deseada y tener que desplazarse a la estacion mas cercana con capacidad. La distancia realizada a pie es 3 veces mayor que la recorrida en bicicleta
    
    Parametros
        move_list : Numpy ndarray
            lista con todos los movimientos realizados. Cada elemento de la lista debe tener la siguiente estructura: [index, desplazamientos]
        init_state : Numpy ndarray
            Contiene el numero de bicicletas alojadas en cada estacion al comienzo de la evaluacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
    Returns
        El numero total de kilometros recorridos por los usuarios
    '''
    tkms = 0 # Kilometros totales
    travel_kms = 0 # Kilometros recorridos por un usuario
    actual_state = init_state.copy() # Variable en la que iremos modificando el numero de biciclestas que se encuentran en cada estacion
    walk_multiplier = 3 # La distancia recorrida andando costara 3 veces mas que la recorrida en bici (valor calculado en kms_m es la distancia en bici)

    for move in move_list:
        station = move[0] # Guardamos la estacion 
        n_bicycle = actual_state[station] + move[1]  # Al numero de bicis de la estacion es cuestion, se le suma el numero de bicis desplazadas (este valor puede ser positivo o negativo) 
        
        # Tenemos que comprobar que el numero de desplazamientos es posible. Es decir, en caso de que se hayan retirado biciletas, que hubiese suficientes para suplir la demanda.
        # Y caso de que se quisiera dejar bicicletas, que existan suficientes slots/espaciones disponibles en la estacion
        if n_bicycle >= 0 and n_bicycle <= capacity[station]: 
            actual_state[station] = n_bicycle # Como el numero obtenido es posible, actualizamos el estado actual de la estacion
        else:
            # En caso de que no existan bicicletas suficientes para suplir la demanda en la estacion
            if n_bicycle < 0:
                actual_state[station] = 0 # La estacion ahora tiene 0 bicicletas
                search = abs(n_bicycle) # abs() nos devuelve el valor absoluto. Obtenemos el valor de bicicletas que necesitamos buscar
                
                # Ahora debemos buscar la estacion mas cercana con capacidad suficiente. Pero podria pasar que una unica estacion no tuviera todos los slots necesarios
                nearest_stations_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'bicycles') # Obtenemos la estacion mas cercana con capacidad suficiente de bicicletas

                for near_station in nearest_stations_index:
                    index = near_station[0]
                    bicycles = near_station[1]

                    # Calculamos los kms recorridos para llegar a la estacion mas cercana
                    travel_kms = kms_m[station][index] * bicycles * walk_multiplier # Kms hacia la estacion * numero de bicicletas a buscar * multiplicador por andar

                    # Actualizamos el estado de la estacion mas cercana
                    nearest_station = index_m[station][index] # Estacion mas cercana a la nuestra
                    actual_state[nearest_station] -= bicycles # Le restamos el numero de bicicletas necesario
                    tkms += travel_kms # Sumamos los kilometros recorridos al total
                        
            # En caso de que no existan slots suficientes para suplir la demanda en la estacion
            elif n_bicycle > capacity[station]:
                actual_state[station] = capacity[station] # Ahora la estacion esta llena
                search = n_bicycle - capacity[station] # Obtenemos el numero de slots a buscar
                
                # Ahora debemos buscar la estacion mas cercana con capacidad suficiente. Pero podria pasar que una unica estacion no tuviera todos los slots necesarios
                nearest_stations_index = find_nearest_station_index(index_m, station, actual_state, capacity, search, 'slots') # Obtenemos la estacion/estaciones mas cercanas con capacidad suficiente de slots

                for near_station in nearest_stations_index:
                    index = near_station[0]
                    slots = near_station[1]

                    # Calculamos los kms recorridos para llegar a la estacion mas cercana
                    travel_kms = kms_m[station][index] * slots # Kms hacia la estacion * numero de slots a buscar

                    # Actualizamos el estado de la estacion mas cercana
                    nearest_station = index_m[station][index] # Estacion mas cercana a la nuestra
                    actual_state[nearest_station] += slots # Le sumamos el numero de bicicletas dejas en los slots libres
                    tkms += travel_kms # Sumamos los kilometros recorridos al total
                    
    return tkms

## Encontrar estación más cercana

In [68]:
def find_nearest_station_index(index_m, station, actual_state, capacity, search, search_type):
    """
    Devuelve una lista con los siguientes valores:
        * index-columna: Index de la estacion mas cercana con respecto al parametro station en la variable index_m
        * slots/bicicletas: Numero de slots/bicicletas aniadidas/quitadas en la estacion index-columna
    
    Parametros
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        station : int
            Estacion desde la que buscar su estacion vecina mas proxima
        actual_state : Numpy ndarray
            Array con el estado actual de bicicletas en cada estacion
        capacity : Numpy ndarray
            Contiene la capacidad maxima de bicicletas que puede almacenar una estacion
        search : int
            Numero de bicicletas/slots a buscar
        search_type : str
            Cadena de caracteres que nos indica el tipo de elemento a buscar: 'bicycles' o 'slots'
            
    Return
        Devuelve una lista con los indices de las estaciones en la matriz index_m y el numero de biciletas/slots cogidos en cada una de las estaciones. Ejemplo:
        [[station_index=3, slots=2], [station_index=12, bicicletas=4]]
    """
    # Valor que almacena la posicion (columna) de la estacion mas cercana con respecto al parametro station
    row_selec = index_m[station] # Seleccionamos el array de estaciones cercanas de la estacion station
    index_lst = np.array([]) # Lista donde guardaremos el index de la estaciones mas cercanas, junto al numero de bicicletas/slots asignados
    remaining = search # Numero de slots que aun faltan por asignar
    
    # Comprobamos que tipo de busqueda estamos realizando, 'bicycles' o 'slots'
    if search_type == 'bicycles':
        # Recorremos el array seleccionado y comprobamos cual es la estacion mas cercana con bicicletas suficientes
        for j in range(1, len(row_selec)):
            # Recorremos el array seleccionado y comprobamos cuales son las estaciones mas cercanas con las bicicletas suficientes
            free_bicycles = actual_state[row_selec[j]]
            
            # Si la estacion tiene biciclestas disponibles se les asigna, en caso contrario comprobamos la siguiente estacion
            if free_bicycles > 0:
                # Cuando el numero de bicicletas de la estacion no suple la demanda
                if remaining > free_bicycles:
                    remaining = remaining - free_bicycles # Actualizamos el numero de bicicletas que faltan por asignar
                    taken = free_bicycles # Debemos tomar estas bicis
                # Cuando el numero de bicicletas de la estacion suple la demanda
                else:
                    taken = remaining # Del numero de bicicletas total disponible. Solo tomamos los necesarios
                    remaining = 0              
                
                index = np.array([[j, taken]]) # Creamos la variable que contiene la estacion y el numero 
                
                # La primera vez que guardamos un index debemos de asignarlo a index_lst
                if len(index_lst) == 0:
                    index_lst = index
                else:
                    index_lst = np.append(index_lst, index, axis=0) # Aniadimos una nueva fila(axis=0) a move_list
                    
                # El bucle termina cuando hemos asignado todos los slots
                if remaining == 0:
                    return index_lst # Devolvemos la lista de movimientos 
            
        raise Exception("Error, ninguna estacion tiene bicicletas")
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            #if search <= actual_state[row_selec[j]]:
            #    return j # Devolvemos el index
            
    elif search_type == 'slots':
        # Recorremos el array seleccionado y comprobamos cuales son las estaciones mas cercanas con los slots suficientes
        for j in range(1, len(row_selec)):
            # Seleccionamos con row_selec la estacion mas cercana a station, despues comprobamos la capacidad disponibles. Y por ultimo comprobamos si es suficiente
            free_slots = capacity[row_selec[j]] - actual_state[row_selec[j]] # Calculamos el numero de slots disponibles restandole a la capacidad total el numero de bicicletas actuales
            
            # Si la estacion tiene slots disponibles se les asigna, en caso contrario comprobamos la siguiente estacion
            if free_slots > 0:
                # Cuando el numero de slots de la estacion no suple la demanda o es igual que la demanda
                if remaining > free_slots:
                    remaining = remaining - free_slots # Actualizamos los slots que faltan por asignar
                    taken = free_slots # Debemos tomar estos slots

                # Cuando el numero de slots de la estacion si suple la demanda
                else:
                    taken = remaining # Del numero de slots total disponible. Solo tomamos los necesarios
                    remaining = 0
                
                index = np.array([[j, taken]])

                # La primera vez que guardamos un index debemos de asignarlo a index_lst
                if len(index_lst) == 0:
                    index_lst = index
                else:
                    index_lst = np.append(index_lst, index, axis=0) # Aniadimos una nueva fila(axis=0) a move_list

                # El bucle termina cuando hemos asignado todos los slots
                if remaining == 0:
                    return index_lst # Devolvemos la lista de movimientos    
        
        raise Exception("Error, todas las estaciones estan llenas")
    else:
        raise Exception("La variable search_type no contiene un tipo valido") # La palabra 'raise' nos permite lanzar una excepcion. Es igual que la palabra 'throw' en otros lenguajes

## Mostrar variación (Función no necesaria)

In [69]:
def show_variation(init_solution, init_tkms, new_solution, new_tkms):
    """
    Muestra por pantalla la diferencia de slots existentes entre dos soluciones

    Parametros
        init_solution : Numpy ndarray
            Solucion inicial a comparar
        init_tkms : float
            Numero de kilometros recorridos en la solucion inicial
        new_solution : Numpy ndarray
            Solucion a comparar con respecto a init_solution
        new_tkms : float
            Numero de kilometros recorridos en la nueva solucion
    """

    columns = np.arange(0,len(init_solution))
    variation = init_solution - new_solution

    # Para representarlo carrectamente la variacion crearemos un string con los espacios correspondientes
    string = '['
    space = ' '
    for i in range(0,len(init_solution)):
        # Es caso de que la variacion sea mayor que 0 y menor a 10, sera necesario aniadir un espacio
        if 10 > variation[i] and variation[i] >= 0:
            if variation [i] == 0: # Cuando aniadimos un 0 debemos agregar un espacio delante. Cuando es un numero positivo, un signo +
                string += space + str(variation[i])
            else:
                string += '+' + str(variation[i])
        else:
            string += str(variation[i])


        if i != len(init_solution)-1:
            string += space

    string += ']'

    print('')
    print('----------------------COMPARACION DE SOLUCIONES-----------------------')
    print('    Estacion    : {}'.format(columns))
    print('----------------------------------------------------------------------')
    print('Solucion inicial: {} - {} km'.format(init_solution, init_tkms))
    print(' Solucion final : {} - {} km'.format(new_solution, new_tkms))
    print('----------------------------------------------------------------------')
    print('    Variacion   : ' + string)
    print('----------------------------------------------------------------------')
    print('')

## Generación de la solución inicial

### GREEDY

Deberemos aumentar el número de cada estación de forma proporcional al número que ya tienen en este momento. Para ello, calcularemos el número total de bicicletas existentes en el estado inicial y dividiremos cada estación por este número. Después aumentaremos estos valores para que la suma total sea igual a 220

In [70]:
def generate_initial_greedy_solution(init_state, max_slots):
    '''
    Crea una solucion inicial para el algoritmo greedy a partir del numero maximo de slots disponibles y el estado inicial
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial de bicletas en las estaciones
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    stations = len(init_state)
    solution = init_state.copy() # Array donde guardaremos la solucion, cada valor del vector representa el numero de slots que tiene
    proportion = max_slots / init_state.sum() # Nueva proporcion para el maximo de slots
    
    # station * proportion
    solution = np.around(solution * proportion).astype(int) # np.around() nos permite redondear los valores del array. Despues le hacemos cast a int
    return solution
    

### EL RESTO DE ALGORITMOS

Crearemos un array con el tamaño de las estaciones, con valor 0. Recorreremos el array y le asignaremos un valor de slots a cada una de forma aleatoria. Para ello, tendremos un rango máximo y mínimo. De forma que, una vez calculado dicho número se hará lo mismo con la siguiente estación. Cuando se llegue a la última estación se le asignara el número restante de slots a repartir

In [71]:
def generate_initial_solution(init_state, max_slots):
    '''
    Crea una solucion a partir del estado inicial y el numero maximo de slots disponibles
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    n_stations = len(init_state)
    
    # 2 3 4 3 2 4 5 2 5
    # 2 3 4 3 2 9 4 1 3
    
    # Usaremos un metodo parecido al usado en la generacion greedy. Generaremos N numeros aleatorios entre min_value y max_value, multiplicamos
    # cada uno por la proporcion entre max_slots/(la suma de todos ellos) y redondear por defecto. 
    
    max_value = round(max_slots/n_stations)
    min_value = 7
    margin = 20 # Margen para hacer que la solucion generada sea compatible con el estado actual
    accept = False # Varible de control
    
    while accept == False:
        solution = np.random.randint(min_value, max_value, n_stations) # Crea un array de numeros aleatorios con el tamanio del numero de estaciones
        # Haremos que la proporcion no este correlacionada exactammente a max_slots, estableceremos un valor mas pequenio para luego poder asignar valores
        # a los las estaciones que no sean compatibles con la solucion inicial
        proportion = (max_slots-margin) / solution.sum() # Nueva proporcion para el maximo de slots
        
        # station * proportion
        solution = np.around(solution * proportion).astype(int) # np.around() nos permite redondear los valores del array. Despues le hacemos cast a int
        amount = solution.sum() 
        remaining = max_slots - amount # Slots restantes por asignar
        
        # Comprobamos a que estacion le faltan slots para satisfacer el estado inicial
        lst = solution < init_state
        for i in range(0,len(lst)):
            if lst[i] == True:
                rest = init_state[i] - solution[i]
                if (remaining - rest) >= 0: 
                    solution[i] += rest # Aniadimos el numero de slots necesarios
                    remaining -= rest # Le quitamos los aniadidos al total restante
                else:
                    break # Si no hay slots suficientes
        
        # Si faltan slots por asignar recorreremos el array asignando slots
        if remaining >= 0:
            # Ahora que tenemos una solucion aceptable aniadimos los slots restantes. Vamos aniadiendo a cada estacion los valores 1 a 1
            i = 0
            while remaining > 0:
                solution[i] += 1 
                remaining -= 1
                i += 1
                if i == n_stations:
                    i = 0
            
            check = solution > init_state
            # Si todas los estaciones de la solucion tienen los slots suficientes para albergar a la solucion inicial. Entonces se acepta
            if check.sum() == n_stations:
                accept = True 

    return solution.astype(int)

In [72]:
# ESTA VERSION NO GENERA UNA SOLUCION COMPATIBLE CON EL ESTADO INICIAL
def generate_solution(n_stations, max_slots):
    '''
    Crea una solucion a partir del numero maximo de slots disponibles. No tiene por que ser compatible con el estado inicial
    
    Parametros
        n_stations : int
            Numero de estaciones que tiene el problema
        max_slots : int
            Numero maximo de slots disponibles a repartir

    Return
        Devuelve un array con el numero de slots disponibles en cada estacion (capacity)
    '''
    # np.random.seed(seed) # Inicializamos la semilla
    solution = np.zeros(16) # Array donde guardaremos la solucion, cada valor del vector representa el numero de slots que tiene
    
    # Debemos establecer un rango de slots que podemos aniadir en cada estacion
    max_value = 30 # Numero minimo de slots que se le puede dar a una estacion
    min_value = 5 # Numero minimo de slots que se le puede dar a una estacion
    assign_slots = 0 # Numero de slots asignados en cada momento
    remaining = max_slots # Slots que faltan por asignar

    for i in range(0,n_stations):
        # Comprobamos el numero de slots que quedan por asignar
        remaining = remaining - assign_slots
        
        # En la estacion debemos asignar todos los slots restantes
        if i != (n_stations-1):
            # Comprobamos que el valor maximo a asignar, es menor que los slots que quedan por repartir
            if remaining < max_value:
                max_value = remaining
                min_value = 0
                
            assign_slots = np.random.randint(min_value, max_value) # Creamos un numero aleatorio
            solution[i] = assign_slots
        
        else:
            solution[i] = remaining
            
    # Ahora necesitamos mezclar las estaciones, ya que es probable que las primeras estaciones tengan mas slots que las ultimas
    np.random.shuffle(solution)

    return solution.astype(int)

## Movement_operator

### Movement_operator Greedy

In [73]:
def greedy_movement_operator(actual_state, actual_solution, neighbours_limit, n_slots):
        """
        Operador de movimiento para el algoritmo greedy. Genera nuevos vecinos a partir de la solucion actual
        
        Parametros
            actual_state : Numpy ndarray
                Estado actual (estado inicial en nuestro problema)
            actual_solution : Numpy ndarray
                Solucion sobre la cual generar nuevos vecinos
            neighbours_limit : int
                Numero de vecinos maximos que se pueden generar. El valor maximo de 120 viene dado por ser el numero maximo de convinaciones posibles entre 16 estaciones
            n_slots : int
                Numero de slots a mover para generar una nueva solucion vecina
        Return
            Lista de soluciones vecinas
        """
        
        n = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles
        neighbours_list = np.array([]) # Lista de vecinos encontrados
        neighbours_n = 0 # Numero de vecinos encontrados
        
        # Recorremos todos los posibles vecinos (movimiento de n_slots entre todas las combinaciones de estaciones posibles)
        for i in range(1, n):
            for j in range(i, n): 
                # Buscamos todos los vecinos posibles que no superen nuestro limite de exploracion
                if neighbours_n < neighbours_limit:
                    # Ahora comprobamos si cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
                    if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
                        neighbours_n += 1 # Aumentamos el numero de vecinos encontrados
                        
                        new_neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
                        new_neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
                        new_neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'
                        
                        if len(neighbours_list) == 0:
                            neighbours_list = np.array([new_neighbour])
                        else:
                            neighbours_list = np.append(neighbours_list, np.array([new_neighbour]), axis=0) # Aniadimos una nueva fila(axis=0) a la lista de vecinos. axis=0 para que la nueva solucion se aniada como fila
                            
        # Devolvemos el mejor vecino
        return neighbours_list


### Movement_operator común

In [74]:
def movement_operator_list(actual_state, actual_solution, n_slots, neighbours_limit, neighbours_map):
    """
    Operador de movimiento que nos da una lista de vecinos. Genera nuevos vecinos a partir de la solucion actual

    Parametros
        actual_state : Numpy ndarray
            Estado actual (estado inicial en nuestro problema)
        actual_solution : Numpy ndarray
            Solucion sobre la cual generar nuevos vecinos
        n_slots : int
            Numero de slots a mover para generar una nueva solucion vecina
        neighbours_limit : int
            Numero de vecinos validos maximo a generar. El valor maximo de 120 viene dado por ser el numero maximo de convinaciones posibles entre 16 estaciones
        neighbours_map : Numpy ndarray
            Lista que contiene todas las combinaciones posibles entre estaciones. Cada posicion de la lista tiene una sublistas con las indices de las estaciones a intercambiar
    Return
        Lista de soluciones vecinas
    """

    n_stations = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles

    # Tenemos que decidir un vecino por el que empezar a explorar. Este vecino sera escogido de forma aleatoria en el mapa de vecinos
    last_neighbours = len(neighbours_map) # Numero total de vecinos
    index = np.random.randint(0, last_neighbours) # Seleccionamos el primer vecino a explorar
    searched = 0 # Contador de total de vecinos visitados

    neighbours_list = np.array([]) # Lista de vecinos encontrados
    neighbours_n = 0 # Numero de vecinos encontrados

    # Exploramos todos los vecinos que podamos hasta llegar al limite de vecinos que podemos explorar (neighbours_limit), si se exploran todos los posibles salimos del bucle
    while neighbours_n < neighbours_limit and searched < last_neighbours:
        i = neighbours_map[index][0] # Estacion i
        j = neighbours_map[index][1] # Estacion j

        # Ahora comprobamos si el cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
        if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
            neighbours_n += 1 # Aumentamos el numero de vecinos encontrados

            new_neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
            new_neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
            new_neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'

            if len(neighbours_list) == 0:
                neighbours_list = np.array([new_neighbour])
            else:
                neighbours_list = np.append(neighbours_list, np.array([new_neighbour]), axis=0) # Aniadimos una nueva fila(axis=0) a la lista de vecinos. axis=0 para que la nueva solucion se aniada como fila

        # Aumentomos la variable index para comprobar el siguiente vecino, comprobar volver al principio del mapa si llegamos al final del mismo
        index += 1
        if index == last_neighbours:
            index = 0 # Hacemos que siga explorando desde el principio del mapa

        # Aumentamos el numero tota de vecinos buscados
        searched += 1

    # Devolvemos la lista de vecinos encontrados
    return neighbours_list


In [75]:
def movement_operator(actual_state, actual_solution, n_slots, neighbours_map, init_index, index):
    """
    Operador de movimiento que nos devuelve un vecino. Usando el mapa de vecinos y la variable init_index nos devuelve el primer vecino valido. Ademas, junto al vecino
    nos devuelve el siguiente vecino a buscar en caso de que el vecino encontrado previamente no satisfada nuestros requisitos. 

    Parametros
        actual_state : Numpy ndarray
            Estado actual (estado inicial en nuestro problema)
        actual_solution : Numpy ndarray
            Solucion sobre la cual generar nuevos vecinos
        n_slots : int
            Numero de slots a mover para generar una nueva solucion vecina
        neighbours_map : Numpy ndarray
            Lista que contiene todas las combinaciones posibles entre estaciones. Cada posicion de la lista tiene una sublistas con las indices de las estaciones a intercambiar
        init_index : int
            Posicion inicial por donde se debe empezar a buscar vecinos en el mapa de vecinos
        index : int
            Siguiente vecino a comprobar
    Return
        Devuelve una lista de dos posiciones. En la primera posicion se guarda el vecino encontrado y en la segunda posicion el siguiente vecino a buscar. En caso de que se haya explorado
        todos los vecinos posibles se devolvera la siguiente lista: [None, -1]. Indicando que el espacio de busqueda ya ha sido explorado
    """

    n_stations = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles

    # Tenemos que decidir un vecino por el que empezar a explorar. Este vecino sera escogido de forma aleatoria en el mapa de vecinos
    last_neighbours = len(neighbours_map) # Numero total de vecinos
    actual_index = index # Seleccionamos el primer vecino a explorar
    neighbour_finded = False # Contador de total de vecinos visitados

    neighbours_n = 0 # Numero de vecinos encontrados
    
    # Exploramos todos los vecinos que podamos hasta llegar al limite de vecinos que podemos explorar (index - 1), si se exploran todos los posibles salimos del bucle
    while neighbour_finded == False and (init_index-1) != actual_index:
        i = neighbours_map[actual_index][0] # Estacion i
        j = neighbours_map[actual_index][1] # Estacion j

        # Ahora comprobamos si el cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
        if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
            neighbour_finded = True # Hemos encontrado un vecino

            neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
            neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
            neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'

        # Aumentomos la variable index para comprobar el siguiente vecino, comprobar volver al principio del mapa si llegamos al final del mismo
        actual_index += 1
        if actual_index == last_neighbours:
            actual_index = 0 # Hacemos que siga explorando desde el principio del mapa

    if (init_index-1) == actual_index:
        neighbour = actual_solution.copy()
        actual_index = -1

    # Devolvemos la lista de vecinos encontrados
    return [neighbour, actual_index]

In [76]:
def create_neigbours_map(n_stations):
    """
    Crea una lista con todas las combinaciones posibles de estaciones. En dicha lista, se guardan los indices de cada estacion
    
    Parametros
        n_stations : int
            Numero de estaciones
    
    Return 
        Devuelve un Numpy ndarray con la siguiente estructura. Lista: [[estacion_i=1, estacion_j=2],[1,3],[1,4], ...]
    """
    neighbours_map = np.array([])
    for i in range(1, n_stations):
        for j in range(i, n_stations):
            if len(neighbours_map) != 0:
                neighbours_map = np.append(neighbours_map,np.array([[i,j]]), axis=0)
            else:
                neighbours_map = np.array([[i,j]])
                
    return neighbours_map

### Tabu Movement Operator

In [77]:
def tabu_movement_operator(actual_state, actual_solution, n_slots, neighbours_map, init_index, index):
    """
    Operador de movimiento que nos devuelve un vecino. Usando el mapa de vecinos y la variable init_index nos devuelve el primer vecino valido. Ademas, junto al vecino
    nos devuelve el siguiente vecino a buscar en caso de que el vecino encontrado previamente no satisfada nuestros requisitos. 

    Parametros
        actual_state : Numpy ndarray
            Estado actual (estado inicial en nuestro problema)
        actual_solution : Numpy ndarray
            Solucion sobre la cual generar nuevos vecinos
        n_slots : int
            Numero de slots a mover para generar una nueva solucion vecina
        neighbours_map : Numpy ndarray
            Lista que contiene todas las combinaciones posibles entre estaciones. Cada posicion de la lista tiene una sublistas con las indices de las estaciones a intercambiar
        init_index : int
            Posicion inicial por donde se debe empezar a buscar vecinos en el mapa de vecinos
        index : int
            Siguiente vecino a comprobar
    Return
        Devuelve una lista de dos posiciones. En la primera posicion se guarda el vecino encontrado y en la segunda posicion el siguiente vecino a buscar. En caso de que se haya explorado
        todos los vecinos posibles se devolvera la siguiente lista: [None, -1]. Indicando que el espacio de busqueda ya ha sido explorado
    """

    n_stations = len(actual_solution) # Guardamos el tamanio de la solucion para hacer los bucles

    # Tenemos que decidir un vecino por el que empezar a explorar. Este vecino sera escogido de forma aleatoria en el mapa de vecinos
    last_neighbours = len(neighbours_map) # Numero total de vecinos
    actual_index = index # Seleccionamos el primer vecino a explorar
    neighbour_finded = False # Contador de total de vecinos visitados
    element = np.array([]) # Lista encargada de almacenar el movimiento realizado. Estructura: [station, station, slots, slots]
    
    # Exploramos todos los vecinos que podamos hasta llegar al limite de vecinos que podemos explorar (index - 1), si se exploran todos los posibles salimos del bucle
    while neighbour_finded == False and (init_index-1) != actual_index:
        i = neighbours_map[actual_index][0] # Estacion i
        j = neighbours_map[actual_index][1] # Estacion j

        # Ahora comprobamos si el cambio de n_slots entra las estaciones 'i' y 'j' es posible realizar (la capacidad de una estacion no puede ser negativa)
        if actual_solution[i] >= n_slots and (actual_solution[i]-n_slots) >= actual_state[i]:
            neighbour_finded = True # Hemos encontrado un vecino

            neighbour = actual_solution.copy() # Creamos una copia de la solucion actual para poder crear la vecina
            neighbour[i] -= n_slots # Le quitamos n_slots a la estacion 'i'
            neighbour[j] += n_slots # Le aniadimos los n_slots quitados de 'i' a 'j'
            
            # Creamos un elemento para la busqueda tabu
            element = np.array([i, j, neighbour[i], neighbour[j]])

        # Aumentomos la variable index para comprobar el siguiente vecino, comprobar volver al principio del mapa si llegamos al final del mismo
        actual_index += 1
        if actual_index == last_neighbours:
            actual_index = 0 # Hacemos que siga explorando desde el principio del mapa

    if (init_index-1) == actual_index:
        neighbour = actual_solution.copy()
        actual_index = -1

    # Devolvemos la lista de vecinos encontrados
    return [neighbour, actual_index, element]

# Algoritmos

## Busqueda aleatoria

El Algoritmo de Búsqueda Aleatoria (BA) consistirá en generar aleatoriamente una solución
en cada iteración debiéndose ejecutar 100 iteraciones con cada semilla devolviendo la mejor de
las iteraciones.

La búsqueda aleatoria completa debe ejecutarse 5 veces, cada vez con una semilla distinta (por
tanto, se deben anotar las 5 semillas que se utilizarán sistemáticamente), para el generador
aleatorio.

### Pseudocódigo del algoritmo
![title](img/random_search-pseudo.png)

In [78]:
def random_search(seed, max_slots, n_slots, init_state, move_list, index_m, kms_m, display):
    """
    Generamos una solucion de forma aleatoria

    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        neighbours_limit : int 
            Numero maximo de vecinos a explorar
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    np.random.seed(seed) # Inicializamos la semilla
    ev = 0 # Numero de evaluaciones realizadas
    
    # Creamos nuestra solucion inicial
    n_stations = len(init_state) # Numero de estaciones
    init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos la solucion mejor solucion que ahora mismo es la inicial
    
    best_solution = init_solution.copy()
    best_tkms = init_tkms
        
    if display == True:
        print('BA init solution: {} - {} kms'.format(init_solution, best_tkms))
    
    # Ahora generamos 100 soluciones diferentes y nos quedamos con la mejor de ellas
    for i in range(0,100):
        actual_solution = generate_initial_solution(init_state, max_slots) # Generamos una nueva solucion
        actual_tkms = evaluate(move_list, init_state, actual_solution, index_m, kms_m) # Evaluamos la solucion actual
        ev += 1 # Aumentamos el numero de evaluaciones
        
        if actual_tkms < best_tkms:
            best_solution = actual_solution.copy()
            best_tkms = actual_tkms
            if display == True:
                print('New BA best solution: {} - {} kms '.format(best_solution, best_tkms))
    
    if display == True:
        show_variation(init_solution, init_tkms, best_solution, best_tkms)
            
    return [best_solution, best_tkms, ev]

## Búsqueda Local

Se implementará el esquema de **el primer mejor** vecino, según el Tema 1 de teoría.

Se partirá de una solución inicial aleatoria. Los algoritmos de búsqueda local tienen su propia condición de parada, pero adicionalmente, en prevención de tiempo excesivos en algún caso, se añadirá una condición de parada alternativa (OR) basada en el número de evaluaciones que esté realizando la búsqueda, es decir, el número de veces que se llame al cálculo de la función de coste. Este valor para la Búsqueda Local será de 3000 llamadas a la función de coste.

La búsqueda con cada algoritmo se debe ejecutar 5 veces con semilla distintas como en el caso de la Aleatoria.

### Pseudocódigo

![title](img/local_search_pseudo.png)

In [79]:
def local_search(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display):
    """
    Devuelve una solucion usando el algoritmo de busqueda local. Se implementa el esquema de primero mejor vecino
        
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        neighbours_limit : int 
            Numero maximo de vecinos a explorar
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    ev = 0 # Numero de evaluaciones realizadas
    
    # Creamos nuestra solucion inicial
    n_stations = len(init_state) # Numero de estaciones
    init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m) # Evaluamos la solucion mejor solucion que ahora mismo es la inicial
    
    if display == True:
        print('LS init solution: {} - {} kms'.format(init_solution, init_tkms))
    
    # Necesitamos crear un mapa de vecinos para el operador de movimiento. Crearemos una lista donde cada posicion representa cada una de las 
    # posibles convinaciones de estaciones posibles. De esta forma, podremos seleccionar una de forma aleatoria y seguir explorando a partir de 
    # dicho vecino
    neighbours_map = create_neigbours_map(n_stations) # Mapa de vecinos
    
    # Inicializamos las variables
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    best_neighbour = None
    best_tkms = None
    
    neighbour_tkms = None
    
    # Variables de control
    max_i = 3000 # Numero maximo de intentos a realizar
    i = 0 # Iteracion actual
    best_neighbour_finded = True
    
    max_possible_neighbours = len(neighbours_map) # Numero maximo de vecinos posibles
    init_index = 0 # Primer vecino a explorar en un vecindario
    actual_index = 0 # Vecino actual a buscar en un vecindario
    
    # Buscamos mientras no superemos el numero maximo de busquedas (max_calls) y siempre que encontremos una solucion mejor a la actual en la nueva iteracion
    while i < max_i and best_neighbour_finded == True:
        # Guardamos el mejor vecino encontrado
        best_neighbour = actual_solution
        best_neighbour_tkms = actual_tkms
        
        # Generamos los vecinos de la solucion actual
        rand = np.random.randint(1,max_possible_neighbours) # Establecemos un vecino por el cual empezar a explorar
        init_index = rand # Vecino inicial explorado
        actual_index = rand # Vecino a explorar
        while True:
            neighbour, actual_index = movement_operator(init_state, actual_solution, n_slots, neighbours_map, init_index, actual_index) # Movement_operator devuelve una lista
            
            # Debemos comprobar que no hemos llegado al final de la lista
            if actual_index != -1: # Si hemos encontrado un nuevo vecino
                neighbour_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m)
                ev += 1 # Aumentamos el numero de evaluaciones

                # Si el vecino encontrado es mejor que nuestra mejor solucion
                if best_neighbour_tkms > neighbour_tkms:
                    best_neighbour = neighbour.copy()
                    best_neighbour_tkms = neighbour_tkms
                    break # Estamos buscando el primer mejor vecino, por lo tanto, salimos del bucle
            else: # Si hemos explorado todos los vecinos dejamos de buscar
                break

        
        # Comprobamos si la solucion obtenida por el vecino es mejor que nuestra solucion actual
        if best_neighbour_tkms < actual_tkms: 
            # Guardamos al vecino como la nueva solucion actual y seguimos buscando
            actual_solution = best_neighbour.copy()
            actual_tkms = best_neighbour_tkms
            if display == True:
                print('New LS best solution: {} - {} kms '.format(actual_solution, actual_tkms))
        else:
            best_neighbour_finded = False
            
        i += 1 # Nueva iteracion
        
    if display == True:
        show_variation(init_solution, init_tkms, actual_solution, actual_tkms)
       
    return [actual_solution, actual_tkms, ev]        
                

## Enfriamiento simulado

Se ha de implementar el algoritmo de Enfriamiento Simulado (ES) con las siguientes
componentes:

* Esquema de enfriamiento: Se implementará el esquema de Cauchy, Tk = T0/(1 + k)
* Condición de enfriamiento L(T): Se enfriará la temperatura, y finalizará a la iteración actual, cuando se haya generado un número máximo de vecinos (independientemente de si han sido o no aceptados).
* Condición de parada: El algoritmo finalizará cuando se alcance un número máximo de iteraciones (enfriamientos).

Se calculará la fórmula inicial mediante la siguiente fórmula:
    
        T0 = (μ / -log(Φ)) * C(Si)

donde T0 es la temperatura inicial, C(Si) es el costo de la solución inicial y Φ[0,1] es la
probabilidad de aceptar una solución un µ por 1 peor que la inicial. En las ejecuciones se
considerar Φ=µ= 0,3.

El número de soluciones generadas en cada temperatura será L(T) = 20 y el número de
enfriamientos (iteraciones) será 80. Se debe repetir también 5 veces con distintas semillas, partiendo de una solución inicial
aleatoria.

![title](img/simulated_annealingV2_pseudo.png)

In [80]:
def simulated_annealing(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display, iterations):
    """
    Devuelve una solucion usando el algoritmo de enfriamiento simulado
        
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        neighbours_limit : int 
            Numero maximo de vecinos a explorar para cada nivel de temperatura
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
        iterations : int
            Cantidad de veces a reducir la temperatura. Una vez que se realicen todas las reducciones el algoritmo termina
            
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    ev = 0 # Numero de evaluaciones realizadas
    
    # Necesitamos crear un mapa de vecinos para el operador de movimiento. Crearemos una lista donde cada posicion representa cada una de las 
    # posibles convinaciones de estaciones posibles. De esta forma, podremos seleccionar una de forma aleatoria y seguir explorando a partir de 
    # dicho vecino
    n_stations = len(init_state)
    neighbours_map = create_neigbours_map(n_stations) # Mapa de vecinos
    max_possible_neighbours = len(neighbours_map) # Numero maximo de vecinos posibles
    
    mu = 0.3 
    phi = 0.3 # Probabilidad de aceptar una solucion
    init_solution = generate_initial_solution(init_state, max_slots)
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m)
    
    init_t = (mu / -np.log10(phi)) * init_tkms # Temperatura inicial
    
    t = init_t # Temperatura actual
    k = 1.380649 * pow(10,-23) # Constante de Boltzmann / pow(base, numero a elevar)
    tk = init_t / (1 + k) # Mecanismo de enfriamiento, esquema de Cauchy
    L = neighbours_limit # Numero de vecinos generados para cada valor de temperatura 
    
    if display == True:
        print('SA init solution: {} - {} kms'.format(init_solution, init_tkms))
    
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    
    # En el algoritmo original tenemos una temperatura y vamos reduciendo su valor. Salimos cuando la temperatura haya disminuido lo suficiente: t >= final_t
    # Nosotros estableceremos un numero de iteraciones maximo
    i = 0
    while i < iterations:
        if display == True:
            print('Iteracion - {} | Temperatura - {}'.format(1+i, t))
            
        # Generamos los vecinos de la solucion actual
        rand = np.random.randint(1,max_possible_neighbours) # Establecemos un vecino por el cual empezar a explorar
        init_index = rand # Vecino inicial explorado
        actual_index = rand # Vecino a explorar
        for j in range(0,L):
            neighbour, actual_index = movement_operator(init_state, actual_solution, n_slots, neighbours_map, init_index, actual_index) # Movement_operator devuelve una lista
            
            # Debemos comprobar que no hemos llegado al final de la lista
            if actual_index != -1: # Si hemos encontrado un nuevo vecino
                neighbour_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m)
                ev += 1 # Aumentamos el numero de evaluaciones
                
                neighbour_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m)
                delta = neighbour_tkms - actual_tkms # Calculo de la diferencia de costos

                 # Aplicamos el criterio de aceptacion
                # Tenemmos que usar un numero uniforme, un valor aleatorio entre 0 y 1
                rand = np.random.rand()
                if (rand < np.exp((-delta) / t)) or (delta < 0):
                    actual_solution = neighbour.copy()
                    actual_tkms = neighbour_tkms
                    #if display == True:
                    #    print('SA init solution: {} - {} kms'.format(actual_solution, actual_tkms))
            else: # Si hemos explorado todos los vecinos dejamos de buscar
                break # Si no se ha encontrado ningun vecino valido debemos dejar de generar vecinos
                
        t = init_t / (i + k) # Reducimos la temperatura
        k += 1 # Elevamos el valor de k para que en la siguiente iteracion se vuelve a reducir la temperatura
        i += 1
    
    if display == True:
        show_variation(init_solution, init_tkms, actual_solution, actual_tkms)
                
    return [actual_solution, actual_tkms, ev]
    

## Búsqueda Tabú

Se implementará la versión de BT utilizando una lista de movimientos tabú y tres estrategias
de reinicialización.

Sus principales características son:

* Estrategia de selección de vecino: Consistirá en examinar 40 vecinos para coger el mejor de acuerdo a los criterios tabú
* Codificación tabú: codificación de valores en cada estación. Cuantas veces toma un valor una estación.
* Selección de estrategias de reinicialización: La probabilidad de escoger la reinicialización construyendo una solución inicial aleatoria es 0,25, la de usar la memoria a largo plazo al generar una nueva solución greedy es 0,5, y la de utilizar la reinicialización desde la mejor solución obtenida es 0,25.
* La solución greedy debe generar soluciones con mayor probabilidad para los valores que menos veces se han producido en cada estación. Para cada estación se deberá calcular las inversas de los valores acumulados y luego normalizar para tener valores entre 0-1 correspondiente a su probabilidad. Se tirará un dado y se elige el primer valor que supera dicho valor añadiéndose a la solución greedy

In [81]:
def tabu_search(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display, iterations):
    """
    Devuelve una solucion usando el algoritmo tabu
        
    Parametros
        seed : int
            Semilla con la que generar los numeros aleatorios
        max_slots : int
            Numero maximo de slots
        n_slots : int
            Numero de slots a mover entre una estacion y otra
        neighbours_limit : int 
            Numero maximo de vecinos a explorar para cada nivel de temperatura
        init_state : Numpy ndarray
            Estado inicial de las estaciones
        move_list : Numpy ndarray
            Lista con todos los movimientos realizados en las estaciones (delta)
        index_m : Numpy ndarray
            Almacena las estaciones mas cercanas a cada estacion ordenadas de mas cercanas a mas lejanas. La fila indica la estacion de refencia, mientras que la columna como de cerca esta la estacion
        kms_m : Numpy ndarray
            Almacena la distancia que hay desde una estacion a otra. La disposicion de la informacion se basa en la variable index_m. Es decir, el valor de cada celda de esta matriz corresponde a la posicion homologa en index_m
        display : bool
            Si es igual a 'True' mostrara por pantalla todas las soluciones que vaya encontrando ademas de una comparativa final entre la primer y la ultima solucion encontrada. Dejar a 'False' para no mostrar nada
        iterations : int
            Cantidad de veces a reducir la temperatura. Una vez que se realicen todas las reducciones el algoritmo termina
            
    Return
        Devuelve una lista con el vector solucion y el numero de kilometros recorridos por los usuarios en dicha solucion. Ademas del numero de evaluaciones - [best_solution, best_tkms, ev]
    """
    # Inicializamos la semilla
    np.random.seed(seed) 
    ev = 0 # Numero de evaluaciones realizadas
    
    # Necesitamos crear un mapa de vecinos para el operador de movimiento. Crearemos una lista donde cada posicion representa cada una de las 
    # posibles convinaciones de estaciones posibles. De esta forma, podremos seleccionar una de forma aleatoria y seguir explorando a partir de 
    # dicho vecino
    n_stations = len(init_state)
    neighbours_map = create_neigbours_map(n_stations) # Mapa de vecinos
    
    slot_limit = 35
    frequency_matrix = create_frequency_matrix(init_state, slot_limit) # Matriz de frecuencias donde se almacenara el numero de veces en las que se ha visitado un vecino
    tabu_max_size = 4 # Tamanio inicial de cada lista tabu
    tabu_list = Tabu(neighbours_map, tabu_max_size)
    
    # Creamos las variables
    init_solution = generate_initial_solution(init_state, max_slots) 
    init_tkms = evaluate(move_list, init_state, init_solution, index_m, kms_m)
    actual_solution = init_solution.copy()
    actual_tkms = init_tkms
    best_solution = actual_solution.copy()
    best_tkms = actual_tkms
    
    max_possible_neighbours = len(neighbours_map) # Numero maximo de vecinos posibles
    init_index = 0 # Primer vecino a explorar en un vecindario
    actual_index = 0 # Vecino actual a buscar en un vecindario
    
    if display == True:
        print('TS init solution: {} - {} kms'.format(init_solution, init_tkms))
    
    # Estrategia de seleccion: consistira en examinar 40 vecinos para coger el mejor de los criterios tabu
    for i in range(0,iterations): 
        '''
        ESTRATEGIA DE REINICIALIZACION
        Debemos reiniciar la lista tabu cada 10 iteraciones. Ademas, debemos aumentar la lista tabu o aumentarla en funcion de un random del 50%
        '''
        if (i % 20) == 0 and i != 0:
            rand = np.random.rand() # Creamos un numero aleatorio
            if rand < 0.5: # Si rand es mejor que 0.5 reducimos el tamanio de la lista
                tabu_max_size -= tabu_max_size
            else: # En caso contrario aumentamos su tamanio
                tabu_max_size += tabu_max_size
                
            # Establicemos un valor minimo posible de elementos tabu
            if tabu_max_size < 4:
                tabu_max_size = 4
                
            tabu_list = Tabu(neighbours_map, tabu_max_size) # Reinicializamos la lista tabu
            
            '''
            Seleccionamos una estrategia de de reinicializacion: La probabilidad de escoger la reinicializacion contruyendo una solucion inicial aleatoria es 0,25,
            la de usar la memoria a largo plazo al generar una nueva solucion greedy es 0,5, y la de utilizar la reinicializacion desde la mejor solucion es 0,25
            '''
            rand = np.random.rand() # Creamos un nuevo numero aleatorio
            if rand < 0.25: # Solucion aleatoria
                actual_solution = generate_initial_solution(init_state, max_slots) # Creamos una nueva solucion aleatoria
                actual_tkms = evaluate(move_list, init_state, actual_solution, index_m, kms_m)
                ev += 1 # Aumentamos el numero de evaluaciones
                
                if display == True:
                    print("Reinilizacion(Iteracion {}) - Solucion Aleatoria".format(i))
            elif rand < 0.5: # Mejor solucion
                actual_solution = best_solution# Seleccionamos la mejor solucion
                actual_tkms = best_tkms
                
                if display == True:
                    print("Reinilizacion(Iteracion {}) - Mejor Solucion".format(i))
            else: # Solucion greedy
                actual_solution = greedy_solution_with(frequency_matrix, init_state) # Solucion greedy
                actual_tkms = evaluate(move_list, init_state, actual_solution, index_m, kms_m)
                ev += 1 # Aumentamos el numero de evaluaciones
                
                if display == True:
                    print("Reinilizacion(Iteracion {}) - Solucion Greedy".format(i))


        '''
        ESTRATEGIA DE SELECCION DE VECINOS
        Ahora debemos explorar los vecinos de la solucion actual
        '''
        best_neighbour = actual_solution.copy()
        best_neighbour_tkms = actual_tkms

        rand = np.random.randint(1,max_possible_neighbours) # Establecemos un vecino por el cual empezar a explorar
        init_index = rand # Vecino inicial explorado
        actual_index = rand # Vecino a explorar
        for j in range(0, neighbours_limit):
            neighbour, actual_index, element = tabu_movement_operator(init_state, actual_solution, n_slots, neighbours_map, init_index, actual_index) # Movement_operator devuelve una lista
            
            # Debemos comprobar que no hemos llegado al final de la lista
            if actual_index != -1: # Si hemos encontrado un nuevo vecino

                # Ahora comprobamos si es vecino tabu
                if not tabu_list.check(element): # Comprobamos si el elemento no es tabu - not es igual que la '!' en Java
                    neighbour_tkms = evaluate(move_list, init_state, neighbour, index_m, kms_m)
                    ev += 1 # Aumentamos el numero de evaluaciones
                    
                    # Si el vecino encontrado es mejor que nuestra mejor vecino
                    if best_neighbour_tkms > neighbour_tkms:
                        best_neighbour = neighbour.copy()
                        best_neighbour_tkms = neighbour_tkms

                        # ALMACENAMOS EL MOVIMIENTO REALIZADO EN LA LISTA TABU
                        if tabu_list.max_size() != 0: # Comprobamos de que podamos aniadir elementos tabu
                            if tabu_list.is_full():
                                tabu_list.remove_old_tabu() # Elimina el elemento tabu mas viejo
                            tabu_list.add(element) # Aniadimos el nuevo elemento
                        
                        # REGISTRAMOS EL MOVIMIENTO REALIZADO EN LA MATRIZ DE FRECUENCIAS
                        frequency_matrix = update_frequency_matrix(frequency_matrix, element)
                        
                        #if display == True:
                        #    print('TS new best neighbour: {} - {} kms'.format(best_neighbour, best_neighbour_tkms))

            else: # Si hemos explorado todos los vecinos dejamos de buscar
                break

        '''
        CRITERIO DE ASPIRACION
        Con el mejor vecino encontrado, comprobamos si es mejor que nuestra mejor solucion
        '''
        if best_neighbour_tkms < best_tkms:
            best_solution = best_neighbour.copy()
            best_tkms = best_neighbour_tkms
                
            if display == True:
                print('    TS new best Solution: {} - {} kms'.format(best_solution, best_tkms))
            
        # Establecemos que el mejor vecino es la nueva solucion actual
        actual_solution = best_neighbour.copy()
        actual_tkms = best_neighbour_tkms
            
               
    if display == True:
        show_variation(init_solution, init_tkms, best_solution, best_tkms)
        
    return [best_solution, best_tkms, ev]

### TABU FUNCIONES Y METODOS AUXILIARES 

#### Clase Tabu

In [82]:
class Tabu:
    """
    Crea una lista de movimientos tabu
    """
    
    def __init__(self, neighbours_map, max_size):
        # El doble guion bajo es para indicar que son variables privadas
        # Comprobamos que max_size no sea negativo
        if max_size < 0:
            max_size = 0
        self._max_size = max_size # Tamanio maximo de la lista tabu
        self._size = 0 # Tamanio de la lista tabu
        self._tabu_dict = self._create_tabu_dict(neighbours_map)
        self._element_list = np.array([]) # Almacenaremos los elementos aniadidos a la lista para conocer su orden y porder borrar los elementos mas viejos
        
        
    def add(self, element):
        """
        Aniade un elemento a la lista Tabu. Este elemento presenta la siguiente estructura:
        
            element = [station, station, value, value]
            
        Parametros
            element : Numpy ndarray
                Elemento a tabu a guardar
        """
        
        if len(element) == 4: # Comprobamos que el objeto elemento tenga el tamanio adecuado
            if self._size < self._max_size:
                key, value = Tabu.convert_element(element)

                state_list = self._tabu_dict.get(key) # Obtenemos la lista de valores
                if len(state_list) <= 0:
                    state_list = np.array([value])
                else:
                    state_list = np.append(state_list, np.array([value]), axis=0)

                self._tabu_dict[key] = state_list # Actualizamos el diccionario
                
                # Actualizamos la lista de elementos
                if len(self._element_list) != 0: # Si mas de una elemento
                    self._element_list = np.append(self._element_list, np.array([element]), axis=0)
                else: # Si no hay elementos creamos la lista
                    self._element_list = np.array([element])
                
                self._size += 1 # Aumentamos el numero de valores almacenados en el diccionario
                
                
            else:
                raise Exception(self.error_size)
        else:
            raise Exception(self.error_element)
            
    def remove_old_tabu(self):
        """
        Elimina el elemento tabu mas antiguo introducido en la lista
        """
        # Comprobamos que haya elementos tabu
        if self._size > 0:
            element = self._element_list[0] # Seleccionamos el elemento mas viejo
            
            '''
            Debemos eliminar el elemento del diccionario
            '''
            key, value = self.convert_element(element) # Obtenemos la clave y el valor
            state_list = self._tabu_dict.get(key) # Obtenemos la lista de valores asociadas a la clave
            for i in range(0, len(state_list)):
                if state_list[i][0] == value[0] and state_list[i][1] == value[1]: # Si ambos valores coinciden
                    new_state = np.delete(state_list, i, 0) # Eliminamos el elemento indicado - delete(array, elemento a borrar en esta caso (i), en que eje quieres borrar - en nuestro caso axis=0 para borrar filas)
                    self._tabu_dict.update({key: new_state}) # Actualizamos la lista de valores asociados a esa clave
                    break # Dejamos de buscar
            
            '''
            Debemos eliminar el elemento de la lista de elementos
            '''
            self._element_list = np.delete(self._element_list, 0, 0) # delete(array, elemento a borrar en esta caso el primero (0), en que eje quieres borrar - en nuestro caso axis=0 para borrar filas)
            
            self._size -= 1 # Reducimos le numero de elementos en la lista tabu
            
        else:
            raise Exception(self.error_empty)
    
    def check(self, element):
        """
        Comprueba si un elemento esta en la list tabu. Este elemento presenta la siguiente estructura:
        
            element = [station, station, value, value]
            
        Parametros
            element : Numpy ndarray
                Elemento a tabu a guardar
                
        Return
            Devuelve True en caso de que si sea un elemento tabu y un False en caso contrario
        """
        check = False
        if len(element) == 4:
            key, value = Tabu.convert_element(element)

            # Si la clave existe
            if key in self._tabu_dict: 
                state_list = self._tabu_dict.get(key)

                # Comprobamos que el valor existe
                if len(state_list) > 0:
                    for state in state_list:
                        if state[0] == value[0] and state[1] == value[1]: # Si ambos valores coinciden
                            check = True # Marcamos que se ha encontrado una coincidencia
                            break # Dejamos de buscar
        else:
            raise Exception(self.error_element)
                        
        return check
    
    def get_value_element(self, element):
        """
        Devuelve todas los combinaciones de valores asigandos a la clave incluida en un elemento.
        
        Parametros
            element : Numpy ndarray
                Lista de 4 valores. Los 2 primeros valores corresponden a las estaciones y los 2 siguiente a los valores. 
                Estructura:
                    element = [station, station, value, value]
                Ejemplo:
                    element = [3,13,4,6]
        
        Return  
            Numpy ndarray
                Lista de combinaciones de valores
        """
        if len(element) == 4:
            stations = np.array([element[0], element[1]])
            return self.get_value_stations(stations)
        else:
            raise Exception(self.error_element)
    
    def get_value_stations(self, stations):
        """
        Devuelve todas los combinaciones de valores asigandos a la clave formada por dos estaciones
        
        Parametros
            element : Numpy ndarray
                Lista de 2 valores. Los 2 valores corresponden a las estaciones
                Estructura:
                    element = [station, station]
                Ejemplo:
                    element = [3,13]
        
        Return 
            Numpy ndarray
                Lista de combinaciones de valores
        """
        if len(stations) == 2:
            key = self.stations_to_key(stations)
            return self.get_value_key(key)
        else:
            raise Exception(self.error_stations)
    
    def get_value_key(self, key):
        """
        Devuelve todas los combinaciones de valores asigandos a una clave([station, station])
        
        Parametros
            stations : Numpy ndarray
                Lista que contiene los 2 valores de las estaciones a buscar
                
        Return  
            Numpy ndarray
                Lista de combinaciones de valores
        """
        if type(key) == str: # str es string
            if key in self._tabu_dict: 
                return self._tabu_dict.get(key).copy()
            else:
                raise Exception(self.error_key)
        else:
            raise Exception(self.error_key_type)
    
    def size(self):
        """
        Devuelve el tamanio de la lista
        
        Return
            Devuelve un entero
        """
        return self._size
    
    def max_size(self):
        """
        Devuelve el tamanio maximo de la lista
        
        Return
            Devuelve un entero
        """
        return self._max_size
    
    def is_full(self):
        """
        Comprueba si la lista tabu esta llena
        
        Return
            Devuelve True si la lista llena y False en caso contrario
        """
        #print('size: {}'.format(self._size))
        # Por como funciona el algoritmo puede pasar que 
        full = False
        if self._max_size == 0: # Si la lista tiene tamanio 0 la lista estara llena
            full = True
        elif self._size >= self._max_size: # Si el tamanio es igual o mayor al tamanio maximo la lista estara llena
            full = True
        
        return full
    
    @staticmethod
    def stations_to_key(stations):
        """
        Convierte una lista de dos elementos en una clave tabu
        
        Parametros
            stations : Numpy ndarray
                Lista de 2 posiciones. Cada valor es igual a la estacion deseada. Estructura: [station, station]. Ejemplo: [3,13]
                
        Return
            string
                Devuelve un string como clave
        """
        a = stations[0] # Estacion 1
        b = stations[1] # Estacion 2
        str_a = '' # String estacion 1
        str_b = '' # String estacion 2
        
        if a < 10:
            str_a = '0' + str(a)
        else:
            str_a = str(a)
            
        if b < 10:
            str_b = '0' + str(b)
        else:
            str_b = str(b)
            
        return str_a + str_b
    
    @staticmethod
    def convert_element(element):
        """
        Convierte una una lista de 4 valores en un par clave/valor
        
        Parametros
            element : Numpy ndarray
                Lista de 4 valores. Los 2 primeros valores corresponden a las estaciones y los 2 siguiente a los valores. 
                Estructura:
                    element = [station, station, value, value]
                Ejemplo:
                    element = [3,13,4,6]
                    
        Return
            key : Numpy ndarray
                Lista de 2 posiciones. El primer valor corresponde a la estacion 0 y la segunda posicion a la estacion 1
            value : Numpy ndarray
                Lista de 2 posiciones. El primer valor corresponded al valor 0 y el segundo valor al valor 1
                
        """
        key = Tabu.stations_to_key(np.array([element[0], element[1]]))
        value = np.array([element[2], element[3]]) 
        return key, value
    
    # El doble guion es para indicar que es un metodo privado
    def _create_tabu_dict(self, neighbours_map):
        """
        Crea un diccionario tabu
        
        Parametros
            neighbours_map : Numpy ndarray
                Contiene todos las combinaciones de estaciones posibles. A partir del mismo, se crearan todas las claves que almacenara el diccionario tabu
        
        Return
            Devuelve un diccionario con cada uno de sus valores igualados a un objeto numpy.array([])
        """
        tabu_dict = {} # Creamos un diccionario
        for stations in neighbours_map:
            # Cada para crear cada clave concatenaremos el valor de las estaciones en una variable string
            key = Tabu.stations_to_key(stations) # Nos devuelve un string con la clave asociada a esas estaciones
            tabu_dict.update({key: np.array([])})
            
        return tabu_dict
    
    # MENSAJES DE ERROR
    error_element = "El elemento no tiene la estructura esperada: [station, station, value, value]"
    error_stations = "Las estaciones no tienen la estructura esperada: [station, station]"
    error_key = "La clave no existe"
    error_key_type = "La clave pasada no es un string"
    error_size = "La lista tabu ya esta llena"
    error_empty = "La lista esta vacia" 
    

#### Metodos auxiliares

In [83]:
def greedy_solution_with(frequency_matrix, init_state):
    """
    Nos devuelve una solucion en base a la matriz de frecuencia. Las convinaciones de estacion/slots menos visitadas tendran mayor probabilidad de aparecer
    
    Parametros
        frequency_matrix : Numpy ndarray
            Con tiene el numero de veces que se ha explorado una convinacion estacion/num. slots
        init_state : Numpy ndarray
            Estado inicial de las estaciones
    
    Return
        Devuelve un vector solucion con las capacidades de cada estacion
    """
    slot_limit = len(frequency_matrix)
    n_stations = len(frequency_matrix[0])
    greedy_solution = np.zeros(n_stations)
    
    '''
    La solucion greedy debe generar soluciones con mayor probabilidad para los valores que menos veces se han
    producido en cada estacion.
    Para cada estacion se debera calcular las inversas de los valores acumulados y luego normalizar para tener
    valores entre 0-1 correspondiente a su probabilidad. Se tirara un dado y se elige el primer valor que supera
    dicho valor aniadiendose a la solucion greedy
    '''
    ones = np.ones((slot_limit, n_stations)) # Matriz de unos
    probability_matrix = ones / frequency_matrix # Matriz que se obtiene al dividir 1 con cada uno valores de la matriz de frecuencias
    
    # Calculamos la suma total de probabilidades para cada estacion
    total_amount = np.zeros(n_stations)
    for i in range(0,n_stations):
        total_amount[i] = sum(probability_matrix[:,i]) # Selecciona todas las filas (slots) de una estacion/columna (i)
        
        # Con la suma total, normalizamos cada valor de probability_matrix
        probability_matrix[:,i] = probability_matrix[:,i] / total_amount[i] # Seleccionamos todos las valores de la estacion y lo dividimos por el valor total
    
    for i in range(0, n_stations): # Estacion
        rand = np.random.rand() # Numero entre 0 y 1
        amount = 0
        for j in range(0, slot_limit): # Numero de slots
            amount += probability_matrix[j][i]
            if rand < amount:
                greedy_solution[i] = j 
                break
                
    # Debemos comprobar que haya un minimo de slots
    total_slot = sum(greedy_solution)
    
    if total_slot < 205:
        i = 0
        while total_slot < 205:
            if i == n_stations:
                i = 0
            greedy_solution[i] += 1 # Aumentamos una cada estacion hasta llegar a 205(maximo de bicis)
            total_slot += 1
            i += 1
            
    if total_slot > 220:
        i = 0
        while total_slot > 220:
            if i == n_stations:
                i = 0
            greedy_solution[i] -= 1
            total_slot -= 1
            i += 1
                
    greedy_solution = greedy_solution.astype(np.int64) # Convertimos el array de float64 a int64
    # Ahora debemos de ajustarla para que sea valida junto al estado inicial
    return adjust_solution(greedy_solution, init_state)
    

In [84]:
def adjust_solution(solution, init_state):
    """
    Ajusta una solucion al estado inicial para que el numero de slots minimo de la solucion coincida con al menos la capacidad del estado inicial
    
    Parametros
        solution : Numpy ndarray
            Vector solucion con las capacidades de las estaciones
        init_state : Numpy ndarray
            Vector con el numero de bicicletas de cada estacion en el estado inicial
            
    Return 
        Devuelve la solucion ajustada al estado inicial
    """
    new_solution = solution.copy()
    #n_stations = len(init_state)
    slots = 4 # Numero de slots a mover
    check_iterations = 15 # Reducimos el numero de slots a mover cada 15 iteraciones
    i = 0 
    check = init_state <= new_solution # Comprobamos que estaciones de la solucion no tienen los slots suficientes
    
    while check.min() == False: # Con check.min() comprobamos si hay alguna estacion que tenga mas bicis que slots
        '''
        Se puede dar el caso en el que moviendo 4 slots nunca se pueda ajustar el vector solution al estado inicial. Por eso, cada cierto numero de 
        iteraciones lo reducimos
        '''
        if ((i % check_iterations) == 0) and (i != 0) and (slots > 1):
            slots -= 1 # Reducimos el numero de slots a mover
        
        # Seleccionamos la estacion que no tenga suficientes slots
        mi = check.argmin() # Estacion con menos vecinos
        
        # Seleccionamos la estacion que tenga mas slots de sobra
        difference = new_solution - init_state
        ma = difference.argmax() # Estacion con mas vecinos
        # Comprobamos que la estacion que necesita vecinos y la que tiene mas vecinos sean distintas
        if ma == mi:
            # Debemos de buscar la segunda estacion con mas slots
            copy = new_solution.copy() # Crearemos una copia para poder encontrar el segundo mejor valor
            copy[ma] = 0 # Establecemos que el valor mas grande ahora tiene 0 slots
            difference = copy - init_state
            ma = difference.argmax() # Estacion con mas vecinos

        
        search = init_state[mi] - new_solution[mi]
        
        # Para no quitar demasiado de una estacion, establecemos un numero maximo de slots a quitar
        if search > slots:
            search = slots
        
        # Le quitamos 1 a la que mas y se lo asignamos a la que menos
        new_solution[ma] -= search
        new_solution[mi] += search
        
        check = init_state <= new_solution # Comprobamos que estaciones de la solucion no tienen los slots suficientes
        i += 1 # Aumentamos el numero de iteraciones
        
    return new_solution
        

In [85]:
def create_frequency_matrix(init_state, slot_limit):
    """
    Crea una matriz de frecuencia inicial. Cada celda de la matriz contiene el numero de veces que se ha explorado una convinacion determinada de estacion(columnas) y numero de slots(filas).
    
    La estructura de la matriz es la siguiente:
    
         ---------------------------------------
         |  Estacion   | 0 | 1 | 2 | 3 | 4 | 5 |
         |--------------------------------------
         | Con 0 slots | 1 | 1 | 1 | 1 | 1 | 1 | 
         | Con 1 slots | 1 | 2 | 2 | 1 | 1 | 4 |
         | Con 2 slots | 3 | 1 | 1 | 5 | 4 | 1 | 
         | Con 3 slots | 1 | 1 | 3 | 4 | 5 | 1 |
         | Con 4 slots | 1 | 6 | 1 | 1 | 1 | 1 |
         | Con 5 slots | 1 | 1 | 4 | 1 | 1 | 7 |
         ---------------------------------------
         
    Por ejemplo, la estacion 0 ha tenido 2 slots en 3 vecinos explorados. Sin embargo, la estacion 5, ha tenido 5 slots en 7 vecinos explorados.
    
    Parametros
        init_state : Numpy ndarray
            Estado inicial de las estaciones al comienzo del algoritmo. Se usara para aumentar cada valor correspondiente del estado en la matriz de frecuencias
        slot_limit : int
            Numero maximo de filas/slots que tendra la matriz. Por ejemplo, en el matriz previamente mostrada. Solo se cuentan como maximo hasta 6 posibles configuraciones de slots. Cuando tiene 0 slota, cuando tiene 1 slots, cuando tiene 2 slots, etc.
    
    Return
        Matriz de frecuencia inicial
    """
    n_stations = len(init_state)
    matrix = np.ones((slot_limit, n_stations)) # Rellenamos todo a uno para luego poder hacer la inversa
    
    # Con la matriz ya creada, procedemos a aumentar las posiciones encontradas en el estado inicial.    
    for i in range(0,n_stations):
        if init_state[i] < slot_limit:
            slot = init_state[i]
        else:
            slot = slot_limit-1
            
        matrix[slot][i] += matrix[slot][i] # Seleccionamos con la columna la estacion, y con la fila el numero de slots que tiene en el estado inicial
        
    return matrix

In [86]:
def update_frequency_matrix(frequency_matrix, element):
    """
    Actualiza la matriz de frecuencias anotando en la misma el movimiento realizado.
    
    Parametros
        frequency_matrix : Numpy ndarray
            Con tiene el numero de veces que se ha explorado una convinacion estacion/num. slots
        element : Numpy ndarray
                Lista de 4 valores. Los 2 primeros valores corresponden a las estaciones y los 2 siguiente a los valores. 
                Estructura:
                    element = [station, station, value, value]
                Ejemplo:
                    element = [3,13,4,6]
            
    Return 
        Devuelve una copia de la matriz de frecuencias actualizada
    """
    
    max_slots, n_stations = frequency_matrix.shape 
    update_matrix = frequency_matrix.copy()
    moves = np.array([[element[0], element[2]], [element[1], element[3]]]) # Un movimiento tiene la siguiente estructura: [station, value]
    
    for move in moves:
        # Comprobamos que el numero de slots maximo no supere el maximo permitido en la matriz de frecuencias. Si lo supera lo ajustamos al maximo
        if move[1] >= max_slots:
            move[1] = max_slots - 1
        
        # update_matrix[value][station]
        update_matrix[int(move[1])][int(move[0])] += 1
    
    return update_matrix
        

# Test

## Greedy test

In [87]:
def greedy_test(n_slots, init_state):
    max_slots = 220 # Numero de slots maximos a repartir
    neighbours_limit = 120 # Numero maximo de vecinos a explorar

    start = time.time()
    greedy_solution = generate_initial_greedy_solution(init_state, max_slots)
    greedy_tkms = evaluate(move_list, init_state, greedy_solution, index_m, kms_m)
    ev = 1 # Solo hemos realizado una evaluacion
    finish = time.time()
    
    total_time = finish-start
    print('    Greedy solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(greedy_solution, greedy_tkms, ev, total_time))
    return [greedy_solution, greedy_tkms, ev, total_time]

## Busqueda aleatoria Test

In [88]:
def random_search_test(seed, n_slots, move_list, init_state, index_m, kms_m, display):
    max_slots = 220 # Numero de slots maximos a repartir
    neighbours_limit = 120 # Numero maximo de vecinos a explorar

    ba_solution = None # Inicializamos la mejor solucion a null
    ba_tkms = None # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    
    start = time.time()
    ba_solution, ba_tkms, ev = random_search(seed, max_slots, n_slots, init_state, move_list, index_m, kms_m, display)
    finish = time.time()   

    total_time = finish-start
    print('    BA best solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(ba_solution, ba_tkms, ev, total_time))
    return [ba_solution, ba_tkms, ev, total_time]

## Busqueda local Test

In [89]:
def local_search_test(seed, n_slots, move_list, init_state, index_m, kms_m, display):
    max_slots = 220 # Numero de slots maximos a repartir
    neighbours_limit = 1 # Numero maximo de vecinos a explorar, devuelveme el primero valido

    ls_solution = None # Inicializamos la mejor solucion a null
    ls_tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    
    start = time.time()
    ls_solution, ls_tkms, ev = local_search(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display)    
    finish = time.time()

    total_time = finish - start
    print('    LS best solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(ls_solution, ls_tkms, ev, total_time))
    return [ls_solution, ls_tkms, ev, total_time]

## Enfriamiento simulado Test

In [90]:
def simulated_annealing_test(seed, n_slots, move_list, init_state, index_m, kms_m, display):
    max_slots = 220 # Numero de slots maximos a repartir
    neighbours_limit = 20 # Numero maximo de vecinos a explorar, devuelveme el primero valido
    iterations = 80 # Numero de iteraciones a realizar

    sa_solution = None # Inicializamos la mejor solucion a null
    sa_tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    
    start = time.time()
    sa_solution, sa_tkms, ev = simulated_annealing(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display, iterations)
    finish = time.time()

    total_time = finish - start
    print('    SA best solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(sa_solution, sa_tkms, ev, total_time))
    return [sa_solution, sa_tkms, ev, total_time]

## Busqueda Tabu Test

In [91]:
def tabu_search_test(seed, n_slots, move_list, init_state, index_m, kms_m, display):
    max_slots = 220 # Numero de slots maximos a repartir
    neighbours_limit = 40 # Numero maximo de vecinos a explorar
    iterations = 80 # Numero de iteraciones a realizar

    ts_solution = None # Inicializamos la mejor solucion a null
    ts_tkms = float('inf') # Igualamos la mejor distancia recorrida a infinito 
    ev = None # Evaluaciones
    
    start = time.time()
    ts_solution, ts_tkms, ev = tabu_search(seed, max_slots, n_slots, neighbours_limit, init_state, move_list, index_m, kms_m, display, iterations)    
    finish = time.time()

    total_time = finish - start
    print('    TS best solution: {} - {} km - Evaluaciones: {} - Tiempo empleado: {} s'.format(ts_solution, ts_tkms, ev, total_time))
    return [ts_solution, ts_tkms, ev, total_time]

# Main

### Generar Semillas

In [92]:
seeds_result_name = '_map'

In [93]:
'''
import numpy as np
import pandas as pd

max_value = 200000000

name = 'seeds' + seeds_result_name + '.csv'
path = './seeds/' + name
number_to_generate = 5
seed_list = None

for i in range(0,number_to_generate):
    rand = np.random.randint(0, max_value)
    if i != 0:
        seed_list = np.append(seed_list, rand)
    else:
        seed_list = np.array([rand])
        
#print('Semillas: {}'.format(seed_list))
df = pd.DataFrame(seed_list)
df.to_csv(path)
'''

"\nimport numpy as np\nimport pandas as pd\n\nmax_value = 200000000\n\nname = 'seeds' + seeds_result_name + '.csv'\npath = './seeds/' + name\nnumber_to_generate = 5\nseed_list = None\n\nfor i in range(0,number_to_generate):\n    rand = np.random.randint(0, max_value)\n    if i != 0:\n        seed_list = np.append(seed_list, rand)\n    else:\n        seed_list = np.array([rand])\n        \n#print('Semillas: {}'.format(seed_list))\ndf = pd.DataFrame(seed_list)\ndf.to_csv(path)\n"

In [203]:
import folium
import numpy as np
import pandas as pd

def create_map(result, init_state):
    """
    Devuelve un objeto mapa a partir de una solucion y del estado inicial
    
    Parametros 
        result : Object
            Resultado extraido de un pandas dataframe
        init_state : Numpy ndarray
    
    Return
        Devuelve un mapa a mostrar
    """
    # Praparamos la solucion
    str_solution = best_solution.iloc[0] 
    str_solution = str_solution.replace('[','')
    str_solution = str_solution.replace(']','')
    result = np.fromstring(str_solution, dtype=int, sep=' ')
    
    # Leemos el archivo de posiciones
    positions = pd.read_csv('./bicicletas/positions.csv', sep=';')
    positions_m = positions.to_numpy()
    
    # Create map object
    l = [43.467935,-3.818877]
    m = folium.Map(location=l, zoom_start=13)
    
    # Range color
    red_n = 0
    color_red = '#e83845'
    green_n = 2
    color_green = '#008000'
    color_blue = '#428bca'
    color = None
    
    # Circle Marker
    n_station = len(init_state)
    r = 1.2 # Radio
    difference = result - init_state # Miramos cuales han aumentado y cual reducido
    for i in range(0,n_station):
        # Seleccionamos color
        if difference[i] < red_n:
            color = color_red,
        elif difference[i] < green_n:
            color = color_green
        else:
            color = color_blue
        
        string = 'Estacion ' + str(i)
        l = positions_m[i,0:2].tolist()
        folium.CircleMarker(
            location = l,
            radius = int(r * abs(result[i])),
            popup = string,
            color = color,
            fill = True,
            fill_color = color
        ).add_to(m)

        
    # Mostamos leyenda por texto
    print("LEYENDA")
    print(f"Color Rojo: Inferior al estado inicial (<{red_n})")
    print(f"Color Verde: Similar al estado inicial (0<n<{green_n})")
    print(f"Color azul: Superior al estado inicial (>{green_n})")
    return m


### Ejecutar

In [129]:
'''
import numpy as np
import pandas as pd
import time

index_df = pd.read_csv('./bicicletas/cercanas_indices.csv')
kms_df = pd.read_csv('./bicicletas/cercanas_kms.csv')
deltas_df = pd.read_csv('./bicicletas/deltas_5m_double.csv')
seeds_df = pd.read_csv('./seeds/seeds'+seeds_result_name+'.csv')
seeds = np.array([])
index_m = index_df.to_numpy() # index matrix
kms_m = kms_df.to_numpy() # kms matrix
move_list = movements_to_list(deltas_df) # Lista de movimientos
init_state = deltas_df.iloc[0].to_numpy() # Estado inicial
seeds = seeds_df.to_numpy()[:,1] # Semillas 

names_list = ['Greedy','Random_search','Local_search','Simulated_annealing','Tabu_search']

n_slots = 1 # Numero de slots a mover
display = False # Muestra el progreso de cada algoritmo en profundidad

# Variables auxiliares
actual_solution = None
actual_tkms = None
ev = None
t = None

solution_list = np.array([]) # Lista de kilometros
tkms_list = np.array([])

df_columns = ['Semilla', 'Algoritmo', 'Kilometros', 'Evaluaciones', 'Solucion'] # Columnas del DataFrame
df = pd.DataFrame(columns=df_columns) # Dataframe que almacena toda la informacion
df_path = './resultados/resultados'+seeds_result_name+'.csv'

'''
#SOLUCION GREEDY
'''
print(names_list[0])
actual_solution, actual_tkms, ev, t = greedy_test(n_slots, init_state)
df = update_dataframe(df, df_columns, 0, names_list[0], actual_tkms, ev, actual_solution)
print('')

'''
#RESTO DE ALGORITMOS
'''
i = 0
for seed in seeds:
    print('SEMILLA({}) - {}'.format(i, seed))
    
    # RANDOM_SEARCH
    print(names_list[1])
    actual_solution, actual_tkms, ev, t = random_search_test(seed, n_slots, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[1], actual_tkms, ev, actual_solution)
    
    # LOCAL_SEARCH
    print(names_list[2])
    actual_solution, actual_tkms, ev, t = local_search_test(seed, n_slots, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[2], actual_tkms, ev, actual_solution)

    # SIMULATED_ANNEALING
    print(names_list[3])
    actual_solution, actual_tkms, ev, t = simulated_annealing_test(seed, n_slots, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[3], actual_tkms, ev, actual_solution)

    # TABU_SEARCH
    print(names_list[4])
    actual_solution, actual_tkms, ev, t = tabu_search_test(seed, n_slots, move_list, init_state, index_m, kms_m, display)
    df = update_dataframe(df, df_columns, seed, names_list[4], actual_tkms, ev, actual_solution)
    
    print('')
    i += 1
    df.to_csv(df_path) # Guardamos el DataFrame
    
print('COMPLETADO')
'''

"\ni = 0\nfor seed in seeds:\n    print('SEMILLA({}) - {}'.format(i, seed))\n    \n    # RANDOM_SEARCH\n    print(names_list[1])\n    actual_solution, actual_tkms, ev, t = random_search_test(seed, n_slots, move_list, init_state, index_m, kms_m, display)\n    df = update_dataframe(df, df_columns, seed, names_list[1], actual_tkms, ev, actual_solution)\n    \n    # LOCAL_SEARCH\n    print(names_list[2])\n    actual_solution, actual_tkms, ev, t = local_search_test(seed, n_slots, move_list, init_state, index_m, kms_m, display)\n    df = update_dataframe(df, df_columns, seed, names_list[2], actual_tkms, ev, actual_solution)\n\n    # SIMULATED_ANNEALING\n    print(names_list[3])\n    actual_solution, actual_tkms, ev, t = simulated_annealing_test(seed, n_slots, move_list, init_state, index_m, kms_m, display)\n    df = update_dataframe(df, df_columns, seed, names_list[3], actual_tkms, ev, actual_solution)\n\n    # TABU_SEARCH\n    print(names_list[4])\n    actual_solution, actual_tkms, ev, t = 

In [204]:
df = pd.read_csv('./resultados/resultados'+seeds_result_name+'.csv')
best = df[df['Kilometros'] == min(df['Kilometros'])]
best_solution = best['Solucion']

create_map(best_solution, init_state)

LEYENDA
Color Rojo: Inferior al estado inicial (<0)
Color Verde: Similar al estado inicial (0<n<2)
Color azul: Superior al estado inicial (>2)
